# Monte Carlo Simulation for FE
## IEOR 4703

### Pricing B-M-S via simulation w/o and w/ Anthithetic Variates

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
# import our BMS price and delta functions
from BMS import BMS_price, BMS_delta

In [2]:
spot = 100
strike = 110
T = 1
sig = 0.30
r = 0.025
q = 0.015

### True value of the call

In [3]:
c = BMS_price('call', spot, strike, r, q, sig, T)
#p = BMS_price('put', spot, strike, r, q, sig, T)
print('call price: ' + str(c))
#print('put price: ' + str(p))

call price: 8.371287380217488


### Pilot program/algorithm

In [4]:
np.random.seed(8943561)

In [5]:
lL = 10000
y1 = np.zeros(lL)
y2 = np.zeros(lL)
z = np.random.randn(lL)
s1 = spot * np.exp((r-q-sig*sig/2)*T + sig*np.sqrt(T)*z)
s2 = spot * np.exp((r-q-sig*sig/2)*T - sig*np.sqrt(T)*z)
y1 = np.maximum(s1 - strike, 0)
y2 = np.maximum(s2 - strike, 0)
cov_y1_y2 = np.sum((y1 - np.mean(y1))*(y2 - np.mean(y2))) / (lL - 1)
# Expecting cov(y1, y2) to be negative
print('cov(y1, y2) = %3.2f' % cov_y1_y2)

cov(y1, y2) = -73.95


### Parameters

In [6]:
n_sims = 100000
m = 50
dt = T/m

### Naive Monte Carlo

In [7]:
# This is inefficient, as for this we do not need to simulate the path,
# we could only simulate the terminal stock price directly
st = time()
payoffs1 = np.zeros(n_sims)
z = np.random.randn(n_sims, m)
s1 = spot * np.prod(np.exp((r-q-sig*sig/2)*dt + sig*np.sqrt(dt)*z), axis=1)
payoffs1 = np.maximum(s1 - strike, 0)
c_hat1 = np.exp(-r * T) * np.mean(payoffs1)
var_hat1 = np.var(payoffs1, ddof=1)
et = time()
print('Elapsed time (w/o) was %3.4f seconds' % (et-st))

Elapsed time (w/o) was 0.2578 seconds


### Monte Carlo with antithetic variates

In [8]:
# this is inefficient, as we can use z's from naive MC, rewrite it!
st = time()
payoffs2 = np.zeros(n_sims//2)
z = np.random.randn(n_sims//2, m)
s1 = spot * np.prod(np.exp((r-q-sig*sig/2)*dt + sig*np.sqrt(dt)*z), axis=1)
s2 = spot * np.prod(np.exp((r-q-sig*sig/2)*dt - sig*np.sqrt(dt)*z), axis=1)
payoffs2 = (np.maximum(s1 - strike, 0) + np.maximum(s2 - strike, 0)) / 2
c_hat2 = np.exp(-r * T) * np.mean(payoffs2)
var_hat2 = np.var(payoffs2, ddof=1)
et = time()
print('Elapsed time (w/) was %3.4f second' % (et-st))

Elapsed time (w/) was 0.1637 second


### Pay attention to the half-width of CI

In [9]:
print('Exact solution: %3.4f' % c)
print('n_sims = %i' % n_sims)
print('Naive MC: %3.4f (%3.4f)' % (c_hat1, 2*np.sqrt(var_hat1/n_sims)))
print('Antithetic MC: %3.4f (%3.4f)' % (c_hat2, 2*np.sqrt(var_hat2/n_sims)))

Exact solution: 8.3713
n_sims = 100000
Naive MC: 8.3837 (0.1156)
Antithetic MC: 8.3812 (0.0724)
